In [1]:
import json
from web3 import Web3
from bip44 import Wallet
from bip44.utils import get_eth_addr

In [2]:
def parse_tx_data(tx_data):
    total_data_len_with_0x = len(tx_data)
    total_data_len_without_0x = total_data_len_with_0x - 2
    total_data_len_bytes = total_data_len_without_0x/2
    print(f"Total data len: {total_data_len_with_0x}({total_data_len_without_0x}) chars")
    print(f"Total data len: {int(total_data_len_bytes)} bytes")
    func_selector = tx_data[0:10]
    print(f"Function selector: {func_selector}")
    tx_data = tx_data[10:]
    arg_i = 0
    while tx_data:
        #print(tx_data)
        arg_hex_value = tx_data[0:64]
        print(f"Arg {arg_i}: {arg_hex_value}")
        tx_data = tx_data[64:]
        arg_i += 1
        

In [3]:
MNEMONIC = "test test test test test test test test test test test junk"
w = Wallet(MNEMONIC)
sk, pk = w.derive_account("eth", address_index=0)
sender = get_eth_addr(pk)
ZERO_ADDRESS = "0x0000000000000000000000000000000000000000"
ETHER = int(1e18)

# Minimal exec fee that is executed automatically by debridge oracles
DEBRIDGE_EXEC_FEE = int(0.02 * ETHER)

In [4]:
with open('abis/DeBridgeGate.json') as json_file:
    debridge_gate_abi = json.load(json_file)

In [5]:
kovan_provider = Web3(Web3.HTTPProvider("https://kovan.poa.network"))
#bsc_testnet_provider = Web3(Web3.HTTPProvider("https://data-seed-prebsc-1-s1.binance.org:8545/"))
bsc_testnet_chain_id = 97

In [6]:
kovan_debridge_gate = kovan_provider.eth.contract(address='0x68D936Cb4723BdD38C488FD50514803f96789d2D', abi=debridge_gate_abi['abi'])

In [7]:
global_fixed_nativeFee = kovan_debridge_gate.functions.globalFixedNativeFee().call()
global_fixed_nativeFee

10000000000000000

In [8]:
# took this from successful Tx initiated via frontend
# this should be creted by pythonoc packer that produces the same result as `packSubmissionAutoParamsTo`
packed_submission_auto_params_to = "0000000000000000000000000000000000000000000000000000000000000020000000000000000000000000000000000000000000000000002386f26fc100000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000008000000000000000000000000000000000000000000000000000000000000000c0000000000000000000000000000000000000000000000000000000000000001476406dbba94f497fa430da5dbd32eda10a3b58860000000000000000000000000000000000000000000000000000000000000000000000000000000000000004d09de08a00000000000000000000000000000000000000000000000000000000"


In [9]:
# contract on the destination network
destination = "0x31649ac584eBcBfD465b49304eB27574B2111F0B"

In [10]:
value = global_fixed_nativeFee + DEBRIDGE_EXEC_FEE
value

30000000000000000

In [11]:
tx = kovan_debridge_gate.functions.send(ZERO_ADDRESS, 30000000000000000, bsc_testnet_chain_id, destination, "", False, 111, packed_submission_auto_params_to).buildTransaction({"from": sender, "nonce": kovan_provider.eth.get_transaction_count(sender),
 "gasPrice": 10_000_000_000, "gas": 1_000_000, "value": value })
tx

{'chainId': 42,
 'from': '0xf39Fd6e51aad88F6F4ce6aB8827279cffFb92266',
 'nonce': 2595,
 'gasPrice': 10000000000,
 'gas': 1000000,
 'value': 30000000000000000,
 'to': '0x68D936Cb4723BdD38C488FD50514803f96789d2D',
 'data': '0xbe2974760000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000006a94d74f4300000000000000000000000000000000000000000000000000000000000000000061000000000000000000000000000000000000000000000000000000000000010000000000000000000000000000000000000000000000000000000000000001400000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000006f0000000000000000000000000000000000000000000000000000000000000180000000000000000000000000000000000000000000000000000000000000001431649ac584ebcbfd465b49304eb27574b2111f0b00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

In [12]:
# This is dirty fix for call data normalization

# we strip redundant argument first (bug in pythonic packer?)

argument_to_remove = 10
txdata = tx['data'][:10+64*argument_to_remove] + tx['data'][10+64*(argument_to_remove+1):]

# replace 0x0180 to 0x0160 (serializer delimiter?)

argument_to_remove = 7
txdata = txdata[:10+64*argument_to_remove] + "0000000000000000000000000000000000000000000000000000000000000160" + txdata[10+64*(argument_to_remove+1):]


# setFee
hex_formatted_value = hex(value)[2:].zfill(64)
argument_to_remove = 1
txdata = txdata[:10+64*argument_to_remove] + hex_formatted_value + txdata[10+64*(argument_to_remove+1):]
parse_tx_data(txdata)


Total data len: 1354(1352) chars
Total data len: 676 bytes
Function selector: 0xbe297476
Arg 0: 0000000000000000000000000000000000000000000000000000000000000000
Arg 1: 000000000000000000000000000000000000000000000000006a94d74f430000
Arg 2: 0000000000000000000000000000000000000000000000000000000000000061
Arg 3: 0000000000000000000000000000000000000000000000000000000000000100
Arg 4: 0000000000000000000000000000000000000000000000000000000000000140
Arg 5: 0000000000000000000000000000000000000000000000000000000000000000
Arg 6: 000000000000000000000000000000000000000000000000000000000000006f
Arg 7: 0000000000000000000000000000000000000000000000000000000000000160
Arg 8: 0000000000000000000000000000000000000000000000000000000000000014
Arg 9: 31649ac584ebcbfd465b49304eb27574b2111f0b000000000000000000000000
Arg 10: 0000000000000000000000000000000000000000000000000000000000000000
Arg 11: 0000000000000000000000000000000000000000000000000000000000000120
Arg 12: 0000000000000000000000000000000000000

In [13]:
# replace tx data with patched one
tx['data'] = txdata

In [14]:
raw_tx = kovan_provider.eth.account.sign_transaction(tx, sk).rawTransaction

In [15]:
tx_hash = kovan_provider.eth.send_raw_transaction(raw_tx)
tx_hash

HexBytes('0xba67767466e9d0eb23663fe5d0754ad555e6c9f512a36e6351fc74ce8bc95fb5')

In [16]:
print("https://testswap.debridge.finance/transaction?tx=" + tx_hash.hex() + "&chainId=42")

https://testswap.debridge.finance/transaction?tx=0xba67767466e9d0eb23663fe5d0754ad555e6c9f512a36e6351fc74ce8bc95fb5&chainId=42


In [18]:
kovan_provider.eth.get_transaction_receipt(tx_hash)

AttributeDict({'blockHash': HexBytes('0xd73511b741e940fd19363876ac36caa3c8ad52ff380715ea753ece1b576f8414'),
 'blockNumber': 28992764,
 'contractAddress': None,
 'cumulativeGasUsed': 107492,
 'effectiveGasPrice': 10000000000,
 'from': '0xf39Fd6e51aad88F6F4ce6aB8827279cffFb92266',
 'gasUsed': 107492,
 'logs': [AttributeDict({'address': '0xd0A1E359811322d97991E03f863a0C30C2cF029C',
   'blockHash': HexBytes('0xd73511b741e940fd19363876ac36caa3c8ad52ff380715ea753ece1b576f8414'),
   'blockNumber': 28992764,
   'data': '0x000000000000000000000000000000000000000000000000006a94d74f430000',
   'logIndex': 0,
   'removed': False,
   'topics': [HexBytes('0xe1fffcc4923d04b559f4d29a8bfc6cda04eb5b0d3c460751c2402c5c5cc9109c'),
    HexBytes('0x00000000000000000000000068d936cb4723bdd38c488fd50514803f96789d2d')],
   'transactionHash': HexBytes('0xba67767466e9d0eb23663fe5d0754ad555e6c9f512a36e6351fc74ce8bc95fb5'),
   'transactionIndex': 0,
   'transactionLogIndex': '0x0',
   'type': 'mined'}),
  AttributeD